In [ ]:
#-------------------------------
# Step 1: Download Data
#-------------------------------
import yfinance as yf
import pandas as pd
from pathlib import Path

# Create folder
EXPORT_DIR = Path("/Users/ikah/Ikah Consulting/Freelance/Portfolio Projects/Portfolio analysis/exports")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# ETFs and date range
TICKERS = ["SPY", "EFA", "IWM", "AGG", "VNQ", "GLD"]
START = "2016-01-01"

# Download adjusted prices
prices = yf.download(TICKERS, start=START, progress=False, auto_adjust=True)

# if multi-level columns appear, select Close prices
if isinstance(prices.columns, pd.MultiIndex):
    prices = prices['Close']
    
prices.head()

# Save daily prices
prices.to_csv(EXPORT_DIR / "daily_prices.csv")

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [ ]:
pio.renderers.default = "iframe_connected"


In [ ]:
import plotly.express as px


In [ ]:
#----------------------------------------
# Step 2: Get Monthly Prices and Returns
#----------------------------------------
# Resample to month-end prices
monthly_prices = prices.resample('ME').last()
monthly_returns = monthly_prices.pct_change().dropna()
monthly_returns.to_csv(EXPORT_DIR / "monthly_returns.csv")
monthly_returns.head()

In [ ]:
#-------------------------------------------------
# Step 3: Equity Curve Chart: Cummulative Returns
#-------------------------------------------------
import plotly.express as px

cum_returns = (1 + monthly_returns['SPY']).cumprod()
df_plot = cum_returns.reset_index()
df_plot.columns = ["Date", "Cumulative Growth"]

fig = px.line(df_plot, x="Date", y="Cumulative Growth", title="Cumulative Growth of SPY (Monthly)")
fig.show()



In [ ]:
#-------------------------------------------------
# Step 4: Multi-EFT Cummulative Returns AND KPIs
#-------------------------------------------------

import numpy as np
import plotly.express as px

# ---- Cumulative returns for all ETFs ----
cum_returns_all = (1 + monthly_returns).cumprod()

# Plot all ETFs together
df_plot_all = cum_returns_all.reset_index()
df_plot_all = pd.melt(df_plot_all, id_vars='Date', var_name='ETF', value_name='Cumulative Growth')

fig = px.line(df_plot_all, x='Date', y='Cumulative Growth', color='ETF',
              title='Cumulative Growth of ETFs (Monthly)')
fig.show()

# ---- Portfolio KPIs ----

def CAGR(returns):
    n = len(returns)/12  # number of years
    return (returns.add(1).prod())**(1/n) - 1

def volatility(returns):
    return returns.std() * np.sqrt(12)  # annualized

def sharpe_ratio(returns, rf=0.03):
    return (CAGR(returns) - rf) / volatility(returns)

def max_drawdown(returns):
    cum = (1 + returns).cumprod()
    peak = cum.cummax()
    drawdown = (cum - peak)/peak
    return drawdown.min()

# Calculate KPIs for each ETF
kpi_list = []
for etf in monthly_returns.columns:
    r = monthly_returns[etf]
    kpi_list.append({
        "ETF": etf,
        "CAGR": CAGR(r),
        "Volatility": volatility(r),
        "Sharpe": sharpe_ratio(r),
        "Max Drawdown": max_drawdown(r)
    })

kpi_df = pd.DataFrame(kpi_list)
kpi_df


In [ ]:
#-------------------------------------------------
# Step 5: Portfolio Level Scenarios/Analysis
#-------------------------------------------------

# Example allocations:
# 1. 60/40 Portfolio (SPY 60%, AGG 40%)
# 2. Custom mix: SPY/EFA/IWM/AGG/VNQ/GLD equally weighted

portfolio_allocations = {
    "60/40": {"SPY": 0.6, "AGG": 0.4},
    "Custom Equal": {etf: 1/len(monthly_returns.columns) for etf in monthly_returns.columns}
}

# ---- Calculate portfolio monthly returns ----
portfolio_returns = pd.DataFrame(index=monthly_returns.index)

for name, weights in portfolio_allocations.items():
    # Ensure all tickers exist in monthly_returns
    weights = {k: v for k, v in weights.items() if k in monthly_returns.columns}
    # weighted sum
    portfolio_returns[name] = monthly_returns[list(weights.keys())].dot(np.array(list(weights.values())))

# ---- Portfolio cumulative returns plot ----
cum_portfolios = (1 + portfolio_returns).cumprod()
df_plot_port = cum_portfolios.reset_index()
df_plot_port = pd.melt(df_plot_port, id_vars='Date', var_name='Portfolio', value_name='Cumulative Growth')

fig = px.line(df_plot_port, x='Date', y='Cumulative Growth', color='Portfolio',
              title='Cumulative Growth of Portfolio Scenarios')
fig.show()

# ---- Portfolio KPIs ----
kpi_portfolio_list = []

for name in portfolio_returns.columns:
    r = portfolio_returns[name]
    kpi_portfolio_list.append({
        "Portfolio": name,
        "CAGR": CAGR(r),
        "Volatility": volatility(r),
        "Sharpe": sharpe_ratio(r),
        "Max Drawdown": max_drawdown(r)
    })

kpi_portfolio_df = pd.DataFrame(kpi_portfolio_list)
kpi_portfolio_df


In [ ]:
from pathlib import Path

EXPORT_DIR = Path("/Users/ikah/Ikah Consulting/Freelance/Portfolio Projects/Portfolio analysis/exports")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# Portfolio returns
portfolio_returns.to_csv(EXPORT_DIR / "portfolio_returns.csv")

# Portfolio KPIs
kpi_portfolio_df.to_csv(EXPORT_DIR / "portfolio_kpis.csv", index=False)


In [ ]:
#-------------------------------------------------
# Step 6: Interactive Model for this notebook.
#-------------------------------------------------
# Includes:
#1. Rolling Volatility
#2. Rolling Sharpe
#3. Drawdowns
#4. User-input custom weights

import plotly.graph_objects as go

# ---- Helper functions ----

def rolling_vol(returns, window=12):
    """Annualized rolling volatility (window in months)"""
    return returns.rolling(window).std() * np.sqrt(12)

def rolling_sharpe(returns, window=12, rf=0.03):
    """Annualized rolling Sharpe ratio"""
    rolling_cagr = (1 + returns.rolling(window).mean())**12 - 1
    rolling_volatility = rolling_vol(returns, window)
    return (rolling_cagr - rf) / rolling_volatility

def drawdown(returns):
    """Compute drawdown series"""
    cum = (1 + returns).cumprod()
    peak = cum.cummax()
    dd = (cum - peak) / peak
    return dd

# ---- User input for custom weights ----
print("Enter custom weights for any of the following ETFs (they will be normalized to sum to 1):")
print(list(monthly_returns.columns))

# Example: user_weights = {"SPY":0.4, "AGG":0.4, "GLD":0.2}
# You can replace this dict to test different allocations
user_weights = {"SPY":0.4, "AGG":0.4, "GLD":0.2}

# Normalize weights
total = sum(user_weights.values())
user_weights = {k: v/total for k, v in user_weights.items() if k in monthly_returns.columns}

# ---- Calculate portfolio returns for user weights ----
custom_portfolio = monthly_returns[list(user_weights.keys())].dot(np.array(list(user_weights.values())))

# ---- Rolling metrics ----
rolling_volatility = rolling_vol(custom_portfolio)
rolling_sharpe_ratio = rolling_sharpe(custom_portfolio)
portfolio_drawdown = drawdown(custom_portfolio)

# ---- Plot rolling metrics ----
fig = go.Figure()

fig.add_trace(go.Scatter(x=rolling_volatility.index, y=rolling_volatility, 
                         mode='lines', name='Rolling Volatility'))
fig.add_trace(go.Scatter(x=rolling_sharpe_ratio.index, y=rolling_sharpe_ratio, 
                         mode='lines', name='Rolling Sharpe'))
fig.add_trace(go.Scatter(x=portfolio_drawdown.index, y=portfolio_drawdown, 
                         mode='lines', name='Drawdown'))

fig.update_layout(title="Portfolio Rolling Metrics & Drawdown",
                  xaxis_title="Date",
                  yaxis_title="Value",
                  legend_title="Metric",
                  template="plotly_white")

fig.show()

# ---- Summary KPIs for the custom portfolio ----
summary_kpis = {
    "CAGR": CAGR(custom_portfolio),
    "Volatility": volatility(custom_portfolio),
    "Sharpe": sharpe_ratio(custom_portfolio),
    "Max Drawdown": max_drawdown(custom_portfolio)
}

print("Custom Portfolio KPIs:")
summary_kpis


In [ ]:
#--------------------------------------------------------
# Step 7: Portfolio Dashboard Model - Interactive Charts.
#--------------------------------------------------------

import plotly.graph_objects as go
import plotly.express as px

# ---- User-defined custom weights ----
# Replace these values to test different allocations
user_weights = {"SPY": 0.5, "EFA": 0.3, "AGG": 0.2}
# Normalize weights
total = sum(user_weights.values())
user_weights = {k: v/total for k, v in user_weights.items() if k in monthly_returns.columns}

# ---- Portfolio scenarios ----
portfolio_allocations = {
    "60/40": {"SPY": 0.6, "AGG": 0.4},
    "Custom Equal": {etf: 1/len(monthly_returns.columns) for etf in monthly_returns.columns},
    "User Custom": user_weights
}

# ---- Calculate portfolio returns ----
portfolio_returns = pd.DataFrame(index=monthly_returns.index)
for name, weights in portfolio_allocations.items():
    weights = {k: v for k, v in weights.items() if k in monthly_returns.columns}
    portfolio_returns[name] = monthly_returns[list(weights.keys())].dot(np.array(list(weights.values())))

# ---- Cumulative returns plot ----
cum_etfs = (1 + monthly_returns).cumprod().reset_index()
cum_portfolios = (1 + portfolio_returns).cumprod().reset_index()

# Melt for Plotly
df_etfs = pd.melt(cum_etfs, id_vars="Date", var_name="ETF", value_name="Cumulative Growth")
df_portfolios = pd.melt(cum_portfolios, id_vars="Date", var_name="Portfolio", value_name="Cumulative Growth")

fig1 = px.line(df_etfs, x="Date", y="Cumulative Growth", color="ETF", title="ETFs Cumulative Growth")
fig2 = px.line(df_portfolios, x="Date", y="Cumulative Growth", color="Portfolio", title="Portfolio Scenarios Cumulative Growth")

fig1.show()
fig2.show()

# ---- Rolling metrics for custom portfolio ----
def rolling_vol(returns, window=12):
    return returns.rolling(window).std() * np.sqrt(12)

def rolling_sharpe(returns, window=12, rf=0.03):
    rolling_cagr = (1 + returns.rolling(window).mean())**12 - 1
    return (rolling_cagr - rf)/rolling_vol(returns, window)

def drawdown(returns):
    cum = (1 + returns).cumprod()
    peak = cum.cummax()
    return (cum - peak)/peak

custom_port = portfolio_returns["User Custom"]
rolling_volatility = rolling_vol(custom_port)
rolling_sharpe_ratio = rolling_sharpe(custom_port)
portfolio_drawdown = drawdown(custom_port)

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=rolling_volatility.index, y=rolling_volatility, name="Rolling Volatility"))
fig3.add_trace(go.Scatter(x=rolling_sharpe_ratio.index, y=rolling_sharpe_ratio, name="Rolling Sharpe"))
fig3.add_trace(go.Scatter(x=portfolio_drawdown.index, y=portfolio_drawdown, name="Drawdown"))
fig3.update_layout(title="Custom Portfolio Rolling Metrics & Drawdown", xaxis_title="Date", yaxis_title="Value", legend_title="Metric", template="plotly_white")
fig3.show()

# ---- Summary KPIs for all portfolios ----
summary_list = []
for col in portfolio_returns.columns:
    r = portfolio_returns[col]
    summary_list.append({
        "Portfolio": col,
        "CAGR": CAGR(r),
        "Volatility": volatility(r),
        "Sharpe": sharpe_ratio(r),
        "Max Drawdown": max_drawdown(r)
    })

kpi_all_portfolios = pd.DataFrame(summary_list)
kpi_all_portfolios


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Example allocation data
allocation = pd.DataFrame({
    'Asset': ['Stock A', 'Stock B', 'Bond C'],
    'Weight': [0.5, 0.3, 0.2],
    'Return': [0.12, 0.08, 0.04]  # Example returns
})

# Pie chart
plt.figure(figsize=(5,5))
plt.pie(allocation['Weight'], labels=allocation['Asset'], autopct='%1.1f%%', startangle=90)
plt.title('Portfolio Allocation')
plt.show()

# Stacked bar: contribution to total return
allocation['Contribution'] = allocation['Weight'] * allocation['Return']
plt.figure(figsize=(6,4))
plt.bar('Portfolio', allocation['Contribution'].sum(), color='skyblue')
plt.bar(allocation['Asset'], allocation['Contribution'], label=allocation['Asset'])
plt.title('Contribution to Portfolio Return')
plt.ylabel('Return Contribution')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Example returns data
returns = pd.DataFrame({
    'Stock A': [0.01, 0.02, -0.01, 0.03],
    'Stock B': [0.02, 0.01, -0.02, 0.01],
    'Bond C': [0.005, 0.004, 0.006, 0.005]
})

# Compute correlation
corr = returns.corr()

# Plot heatmap with Matplotlib
plt.figure(figsize=(6,5))
plt.imshow(corr, cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Correlation')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title('Correlation Heatmap')
plt.show()
